<a href="https://colab.research.google.com/github/rprimi/colB5BERT/blob/main/python/colB5BERT_fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **colB5BERT:** Fine tuning colBERT with Big Five dataset

* Notebook inspirado no buscador denso de Marcos Pio

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip3 install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 51.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 112.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 85.5 MB/s eta 0:00:00


In [4]:
!git clone https://github.com/rprimi/colB5BERT.git

%cd /content/colB5BERT
!git pull



Cloning into 'colB5BERT'...
remote: Enumerating objects: 238, done.
remote: Counting objects: 100% (42/42), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 238 (delta 31), reused 18 (delta 18), pack-reused 196
Receiving objects: 100% (238/238), 32.66 MiB | 32.13 MiB/s, done.
Resolving deltas: 100% (146/146), done.
/content/colB5BERT
Already up to date.


### Libraries

In [4]:
import sys
sys.path.append('/content/colB5BERT/python/')

import utils
import vsm
import sst




In [5]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

/bin/bash: nvidia-smi: command not found


In [ ]:
b5_data = pd.read_csv('/content/colB5BERT/data/db_textos.splitted.csv', sep=';')
b5_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11537 entries, 0 to 11536
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              11537 non-null  int64 
 1   id_divisao      11537 non-null  int64 
 2   texto_dividido  11537 non-null  object
dtypes: int64(2), object(1)
memory usage: 270.5+ KB


In [ ]:
import pandas as pd
base_itens_b5 = pd.read_excel('/content/colB5BERT/data/base_itens.xlsx')


base_itens_b5
base_itens_b5.info()
# base_itens_b5['item_pt_text'].tolist()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 415 entries, 0 to 414
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ord0_index    415 non-null    int64  
 1   test          415 non-null    object 
 2   coditem       415 non-null    object 
 3   item_pt_text  415 non-null    object 
 4   item_en_text  415 non-null    object 
 5   domain        413 non-null    object 
 6   facet         413 non-null    object 
 7   pole          415 non-null    int64  
 8   seman_pairs   273 non-null    float64
dtypes: float64(1), int64(2), object(6)
memory usage: 29.3+ KB


In [ ]:
bert_weights_name = 'neuralmind/bert-base-portuguese-cased'
bert_tokenizer = BertTokenizer.from_pretrained(bert_weights_name)
bert_model = BertModel.from_pretrained(bert_weights_name)

Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
